In [1]:
# Modues to be used
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Load data from pickle files

# -> Player dictionary with the dataFrames
f = open('data/dict_player.pickle', 'rb')
dict_player = pickle.load(f)

# -> Map dataFrames
f = open('data/df_map.pickle','rb')
df_map = pickle.load(f)

# -> Map dictionary
f = open('data/dict_map.pickle','rb')
dict_map = pickle.load(f)

# Save a list of the teams and their ranks

In [8]:
df_map.head()
df_map.keys()

Index(['map_id', 'map', 'date', 'event_id', 'event_name', 'team_id_1',
       'team_name_1', 'team_score_1', 'team_kills_1', 'team_deaths_1',
       'team_assists_1', 'team_rank_1', 'team_id_2', 'team_name_2',
       'team_score_2', 'team_kills_2', 'team_deaths_2', 'team_assists_2',
       'team_rank_2', 'winner'],
      dtype='object')

In [90]:
df_aux = df_map[ df_map['date']>datetime(2019,1,1) ]

df_1 = df_aux.drop(columns=['map', 'event_id', 'event_name',
       'team_score_1', 'team_kills_1', 'team_deaths_1',
       'team_assists_1','team_id_2', 'team_name_2',
       'team_score_2', 'team_kills_2', 'team_deaths_2', 'team_assists_2',
       'team_rank_2', 'winner']).reset_index()

df_1 = df_1.rename(columns={'team_id_1':'team_id', 'team_name_1':'team_name','team_rank_1':'team_rank'})


df_2 = df_aux.drop(columns=[ 'map', 'event_id', 'event_name',
       'team_score_1', 'team_kills_1', 'team_deaths_1',
       'team_assists_1','team_id_1', 'team_name_1',
       'team_score_2', 'team_kills_2', 'team_deaths_2', 'team_assists_2',
       'team_rank_1', 'winner']).reset_index()

df_2 = df_2.rename(columns={'team_id_2':'team_id', 'team_name_2':'team_name','team_rank_2':'team_rank'})

df = df_1.append(df_2,sort=True,ignore_index=True)

df = df.sort_values('date',ascending=False)

team_id_vec = df.team_id.unique()

dict_team = {}
for team_id in team_id_vec:
    
    df_tmp = df[ df['team_id'] == team_id].head(n=1)

    dict_team[team_id] = {}
    dict_team[team_id]['team_name'] = df_tmp.team_name.values[0]
    dict_team[team_id]['team_rank'] = df_tmp.team_rank.values[0]
    dict_team[team_id]['team_prize'] = 0.0

    

    map_id = df_tmp.map_id.values[0]

    count = 0
    for player_id in dict_map[map_id][team_id]['players_id']:

        count = count + 1
        dict_team[team_id]['player_'+str(count)] = dict_player[player_id]['player_name'].values[0]

        dict_team[team_id]['team_prize'] = dict_team[team_id]['team_prize'] + dict_player[player_id][ dict_player[player_id]['date']>datetime(2019,1,1) ]['prize'].sum() / 6.0

    if dict_team[team_id]['team_prize'] > 0.0:
        dict_team[team_id]['team_prize_rating'] = np.log( dict_team[team_id]['team_prize'] ) / 12.0
    else:
        dict_team[team_id]['team_prize_rating'] = 0.0



In [91]:
dict_team[6665]

{'team_name': 'Astralis',
 'team_rank': 1,
 'team_prize': 299791.6666666667,
 'player_1': 'Xyp9x',
 'player_2': 'device',
 'player_3': 'gla1ve',
 'player_4': 'dupreeh',
 'player_5': 'Magisk',
 'team_prize_rating': 1.050903588996305}

In [92]:
df_map

,map_id,map,date,event_id,event_name,team_id_1,team_name_1,team_score_1,team_kills_1,team_deaths_1,team_assists_1,team_rank_1,team_id_2,team_name_2,team_score_2,team_kills_2,team_deaths_2,team_assists_2,team_rank_2,winner
12838,12838,NaN,2012-09-13 19:00:00,965,Go4CS:GO Cup #6,4411,NiP,16,91.0,46.0,0.0,0,4443,PRiME,3,45.0,91.0,0.0,0,1
12839,12839,NaN,2012-09-13 19:00:00,965,Go4CS:GO Cup #6,4411,NiP,16,83.0,34.0,0.0,0,4443,PRiME,1,34.0,83.0,0.0,0,1
12840,12840,NaN,2012-09-13 19:00:00,966,CyberGamer Qualifier Cup #1 by Tt eSPORTS,4444,Blight,16,81.0,37.0,0.0,0,4445,FMESPORTS,2,37.0,81.0,0.0,0,1
12841,12841,NaN,2012-09-13 19:00:00,966,CyberGamer Qualifier Cup #1 by Tt eSPORTS,4444,Blight,16,97.0,58.0,0.0,0,4445,FMESPORTS,7,57.0,97.0,0.0,0,1
12848,12848,NaN,2012-09-23 12:00:00,973,DreamHack Valencia 2012,4411,NiP,16,102.0,86.0,0.0,0,4467,VeryGames,10,86.0,102.0,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97591,97591,trn,2020-01-10 00:35:00,5104,IEM Katowice 2020 North America Closed Qualifier,5752,Cloud9,13,90.0,105.0,12.0,18,9215,MIBR,16,105.0,90.0,20.0,14,2
97592,97592,nuke,2020-01-10 01:00:00,5164,Aorus League 2019 #4 Northern Cone,10330,Supremacy,16,98.0,69.0,17.0,0,10578,Infamous,8,69.0,98.0,13.0,0,1
97593,97593,ovp,2020-01-10 00:35:00,5104,IEM Katowice 2020 North America Closed Qualifier,5752,Cloud9,16,77.0,46.0,11.0,18,9215,MIBR,4,46.0,77.0,10.0,14,1
97594,97594,inf,2020-01-10 01:00:00,5164,Aorus League 2019 #4 Northern Cone,10330,Supremacy,16,99.0,93.0,16.0,0,10578,Infamous,12,93.0,100.0,17.0,0,1


In [87]:
dict_player[5973].head()

KeyError: 5973